## PuLP formulation for the FSGSP

### Importing modules

In [2]:
from pulp import *
import cplex

### Defining the PuLP-FSGSP function

In [3]:
def fsgsp_pulp(groups, jobs_groups, num_jobs, machines, 
               num_jobs_k, run_time, setup_time, M=999999):
    
    # Defining the problem as a minimisation problem
    prob = LpProblem("FlowshopGroupScheduling", sense=LpMinimize)
    
    # Defining the variables
    # (X_lj) job j is processed after job l
    X_job_sequence = LpVariable.dicts("X", ((l, j)\
                                           for k in groups\
                                           for j in jobs_groups[k - 1]\
                                           for l in jobs_groups[k - 1]\
                                           if j > l),
                                      cat="Binary")
        
    # (U_tk) group k is processed immediately after group t
    U_group_sequence = LpVariable.dicts("U", ((t, k)\
                                             for t in range(len(groups) + 1)\
                                             for k in groups\
                                             if t != k), 
                                        cat="Binary")
    
    # (F_ki) the finishing time of the last job of group k on machine i
    F_finishing_group = LpVariable.dicts("F", ((k, i)\
                                              for k in range(len(groups) + 1)\
                                              for i in machines), lowBound=0,\
                                              upBound=None, 
                                         cat="Continuous")
    # (S_ki) the starting time of the first job of group k on machine i
    S_starting_group = LpVariable.dicts("S", ((k, i)\
                                             for k in groups\
                                             for i in machines), 
                                        lowBound=0, upBound=None, cat="Continuous")
    
    # (C_ji) the completion time of job j on machine i
    C_completion_job = LpVariable.dicts("C", ((j, i)\
                                             for k in groups\
                                             for j in jobs_groups[k - 1]\
                                             for i in range(len(machines) + 1)),
                                        lowBound=0, upBound=None, cat="Continuous")
    
    # Defining the objective function for the problem: the total completion time of jobs
    prob += lpSum(C_completion_job[j, machines[-1]] for k in groups for j in jobs_groups[k - 1])
    
    # Defining the constraints of the problem
    # (7) Completion time of a job on a machine should be greater than its completion time on the previous machine
    for k in groups:
        for j in jobs_groups[k - 1]:
            for i in machines:

                if i == 1:
                    C_completion_job[j, i - 1] = 0

                prob += C_completion_job[j, i] >= (C_completion_job[j, i - 1] + run_time[j - 1][i - 1])

    # (8 and 9) Jobs are not processed simultaneously on each machine
    for k in groups:
        for j in jobs_groups[k - 1]:
            for l in jobs_groups[k - 1]:
                if j > l:
                    for i in machines:
                        # (8)
                        prob += C_completion_job[j, i] >= (C_completion_job[l, i] +\
                                                           run_time[j - 1][i - 1] -\
                                                           (1 - X_job_sequence[l, j]) * M)
                        # (9)
                        prob += C_completion_job[l, i] >= (C_completion_job[j, i] +\
                                                          run_time[l - 1][i - 1] -\
                                                          (X_job_sequence[l, j] * M))

    # (10 and 11) Jobs should start and finish between starting and finishing times of the group it belongs to
    for k in groups:
        for j in jobs_groups[k - 1]:
            for i in machines:
                # (10)
                prob += C_completion_job[j, i] >= (S_starting_group[k, i] +\
                                                  run_time[j - 1][i - 1])
                # (11)
                prob += F_finishing_group[k, i] >= C_completion_job[j, i]

    # (12) Each group on each machine starts processing after last job of the immediately previous group is finished
    for t in range(len(groups) + 1):
        for k in groups:
            if t != k:
                for i in machines:

                    if t == 0:
                        F_finishing_group[t, i] = 0

                    prob += S_starting_group[k, i] >= (F_finishing_group[t, i] +\
                                                       setup_time[i - 1][t][k - 1] -\
                                                       ((1 - U_group_sequence[t, k]) * M))

    # (13) Each group should have exactly one group as the previous processed group on each machine
    for k in groups:

        prob += lpSum(U_group_sequence[t, k] for t in range(len(groups) + 1)\
                                             if t  != k) == 1

    # (14) Each group has at most one successor group
    for t in range(len(groups) + 1):

        prob += lpSum(U_group_sequence[t, k] for k in groups if t != k) <= 1

    # (15) The reference group -group zero- is assigned to the first slot
    prob += lpSum(U_group_sequence[0, k] for k in groups) == 1

    # (16) Each group can be processed before or after another group
    for t in groups[:-1]:
        k = t + 1

        prob += U_group_sequence[t, k] + U_group_sequence[k, t] <= 1
    
    # Solving the problem using the CPLEX solver
    prob.solve(solver=CPLEX_PY())
    
    # Showing the results of the optimisation process
    print("Status: ", LpStatus[prob.status])
    
    # Showing the values of the variables
    for v in prob.variables():
        print(v.name, " = ", v.varValue)
        
    # Showing the value of the objective function
    print("Objective function value: ", value(prob.objective))
    
    return value(prob.objective)

### A randomly generated example for the FSGSP

This example is randomly generated. The number of groups is generated between 2 and 5, the number of jobs within each group is between 2 and 4, 2 machines comprise the manufacturing cell, run times of each job on each machine are between 1 and 20, and setup times of each group processed after another group on each machine is between 1 and 50.

In [4]:
import numpy as np

np.random.seed(90)

# Parameters
# (g) the number of groups
groups = list(range(1, np.random.randint(2, 5 + 1, 1)[0] + 1)) # Small

# (G_k) a set including the jobs belonging to group k
jobs_groups = []
jobs_count = 1
for gr in groups:
    n_jobs = np.random.randint(2, 4 + 1, 1)[0] # Small
    jobs = list(range(jobs_count, jobs_count + n_jobs))
    jobs_groups.append(jobs)
    jobs_count += n_jobs


# (j) the number of jobs
num_jobs = jobs_groups[-1][-1]

# (m) the number of machines
machines = [1, 2]

# (n_k) the number of jobs in group k
num_jobs_k = []  # [2, 3]
for g in jobs_groups:
    num_jobs_k.append(len(g))
    
# (P_ji) run time of job j on machine i -> run_time[job - 1][machine - 1]
run_time = []
for job in range(num_jobs):
    job_list = []
    for mach in machines:
        job_list.append(np.random.randint(1, 20, 1)[0])
    run_time.append(job_list)

# (a_tki) setup time of group k processed after group t on machine i (small problem - Level 1 - 2-machine)
# -> setup_time[machine-1][prec group][suc group - 1] 
setup_time = []
for machine in machines:
    mach_list = []
    for gr in range(len(groups) + 1):
        group_list = []
        for g in groups:
            if gr == g:
                group_list.append(0)
            else:
                if machine == 1:
                    group_list.append(np.random.randint(1, 50, 1)[0])
                elif machine == 2:
                    group_list.append(np.random.randint(1, 50, 1)[0])
                elif machine == 3:
                    group_list.append(np.random.randint(1, 50, 1)[0])
        mach_list.append(group_list)
    setup_time.append(mach_list)

## (M) a large positive number
M = 999999

In [8]:
print("The groups or part-families: ", groups)
print("The jobs within each group: ", jobs_groups)
print("The run times of each job on each machine:\n", run_time)
print("The setup times of each group processed after another group on each machine:\n", setup_time)

The groups or part-families:  [1, 2, 3, 4, 5]
The jobs within each group:  [[1, 2, 3], [4, 5, 6, 7], [8, 9], [10, 11, 12, 13], [14, 15, 16]]
The run times of each job on each machine:
 [[19, 11], [1, 12], [17, 19], [14, 19], [10, 1], [1, 11], [9, 17], [12, 13], [5, 10], [17, 7], [12, 17], [16, 2], [15, 3], [3, 9], [11, 3], [17, 19]]
The setup times of each group processed after another group on each machine:
 [[[2, 33, 8, 23, 7], [0, 45, 14, 3, 2], [7, 0, 46, 34, 14], [13, 8, 0, 44, 34], [20, 12, 43, 0, 39], [10, 43, 3, 41, 0]], [[5, 35, 44, 46, 39], [0, 24, 41, 47, 47], [15, 0, 44, 21, 4], [28, 12, 0, 18, 35], [32, 32, 41, 0, 1], [37, 48, 14, 40, 0]]]


### Calling the function and solving the problem

In [5]:
fsgsp_pulp(groups, jobs_groups, num_jobs, machines,
           num_jobs_k, run_time, setup_time)

Version identifier: 20.1.0.0 | 2020-11-11 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 17 rows and 0 columns.
MIP Presolve modified 20 coefficients.
Reduced MIP has 221 rows, 96 columns, and 586 nonzeros.
Reduced MIP has 44 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (0.24 ticks)
Found incumbent of value 3111.000000 after 0.11 sec. (1.03 ticks)
Probing time = 0.00 sec. (0.06 ticks)
Cover probing fixed 0 vars, tightened 120 bounds.
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 6 rows and 0 columns.
MIP Presolve modified 122 coefficients.
Reduced MIP has 215 rows, 96 columns, and 574 nonzeros.
Reduced MIP has 44 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.32 ticks)
Probing fixed 0 vars, tightened 56 bounds.
Probing time = 0.01 sec. (0.25 ticks)
Clique table members: 15.
MIP emphasis: balance optimality and feasibility.
MIP search met

2482.999999999998

For a randomly generated example of the FSGSP, using the seed 90 `np.random.seed(90)`, the CPLEX solver **finds an optimal solution** with an objective function value of **2482**. The sequence of groups is [1, 4, 2, 5, 3] and the sequence of jobs is [2, 3, 1] for group 1, [5, 6, 7, 4] for group 2, [9, 8] for group 3, [12, 13, 10, 11] for group 4 and [15, 14, 16] for group 5.